# Exercise with Natural Language Processing

For todays exersice we will be doing two things.  The first is to build the same model with the same data that we did in the lecture, the second will be to build a new model with new data. 

## PART 1: 
- 20 Newsgroups Corpus


## PART 2:
- Republican vs Democrat Tweet Classifier

In [60]:
# Import pandas for data handling
import pandas as pd

# NLTK is our Natural-Language-Took-Kit
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Libraries for helping us with strings
import string
# Regular Expression Library
import re

# Import our text vectorizers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


# Import our classifiers
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier


# Import some ML helper function
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report


# Import our metrics to evaluate our model
from sklearn import metrics
from sklearn.metrics import classification_report


# Library for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# You may need to download these from nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\conor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\conor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\conor\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Load and display data.
1. Load the 20-newsgroups.csv data into a dataframe.
1. Print the shape
1. Inspect / remove nulls and duplicates
1. Find class balances, print out how many of each topic_category there are.

In [61]:
# 1. Load the 20-newsgroups.csv data into a dataframe.
# 2. Print the shape
df = pd.read_csv('data/20-newsgroups.csv')
print(df.shape)
df.head()

(11314, 4)


,id,message,topic,topic_category
0,0,From: lerxst@wam.umd.edu (where's my thing)\r\...,7,rec.autos
1,1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,3,From: jgreen@amber (Joe Green)\r\nSubject: Re:...,1,comp.graphics
4,4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [62]:
# 3. Inspect / remove nulls and duplicates
df.isnull().sum()

id                0
message           0
topic             0
topic_category    0
dtype: int64

In [63]:
df.duplicated().sum()

0

In [64]:
# 4. Find class balances, print out how many of each topic_category there are.
df.topic_category.value_counts()

rec.sport.hockey            600
soc.religion.christian      599
rec.motorcycles             598
rec.sport.baseball          597
sci.crypt                   595
rec.autos                   594
sci.med                     594
comp.windows.x              593
sci.space                   593
comp.os.ms-windows.misc     591
sci.electronics             591
comp.sys.ibm.pc.hardware    590
misc.forsale                585
comp.graphics               584
comp.sys.mac.hardware       578
talk.politics.mideast       564
talk.politics.guns          546
alt.atheism                 480
talk.politics.misc          465
talk.religion.misc          377
Name: topic_category, dtype: int64

# Text Pre-Processing 
(aka Feature engineering)
1. Make a function that makes all text lowercase.
    * Do a sanity check by feeding in a test sentence into the function. 
    
    
2. Make a function that removes all punctuation. 
    * Do a sanity check by feeding in a test sentence into the function. 
    
    
3. Make a function that removes all stopwords.
    * Do a sanity check by feeding in a test sentence into the function. 
    
    
4. EXTRA CREDIT (This step only): Make a function that stemms all words. 


5. Mandatory: Make a pipeline function that applys all the text processing functions you just built.
    * Do a sanity check by feeding in a test sentence into the pipeline. 
    
    
    
6. Mandatory: Use `df['message_clean'] = df[column].apply(???)` and apply the text pipeline to your text data column. 

In [65]:
# 1. Make a function that makes all text lowercase.
def make_lower(a_string):
    return a_string.lower()
test_string = 'This is A SENTENCE with LOTS OF CAPS.'
test_string = make_lower(test_string)
print(test_string)

this is a sentence with lots of caps.


In [66]:
# 2. Make a function that removes all punctuation. 
def remove_punctuation(a_string):
    a_string = re.sub(r'[^\w\s]','',a_string)
    return a_string

test_string = 'This is a sentence! 50 With lots of punctuation??? & other #things.'
test_string = remove_punctuation(test_string)
print(test_string)

This is a sentence 50 With lots of punctuation  other things


In [67]:
# 3. Make a function that removes all stopwords.
def remove_stopwords(a_string):
    # Break the sentence down into a list of words
    words = word_tokenize(a_string)
    
    # Make a list to append valid words into
    valid_words = []
    
    # Loop through all the words
    for word in words:
        
        # Check if word is not in stopwords
        if word not in stopwords:
            
            # If word not in stopwords, append to our valid_words
            valid_words.append(word)

    # Join the list of words together into a string
    a_string = ' '.join(valid_words)

    return a_string

test_string = 'This is a sentence! With some different stopwords i have added in here.'
test_string = remove_stopwords(test_string)
print(test_string)

This sentence ! With different stopwords added .


In [68]:
# 4. EXTRA CREDIT: Make a function that stemms all words. 
def stem_words(a_string):
     # Initalize our Stemmer
    porter = PorterStemmer()
    
     # Break the sentence down into a list of words
    words = word_tokenize(a_string)
    
     # Make a list to append valid words into
    valid_words = []

     # Loop through all the words
    for word in words:
         # Stem the word
        stemmed_word = porter.stem(word)
        
         # Append stemmed word to our valid_words
        valid_words.append(stemmed_word)
        
     # Join the list of words together into a string
    a_string = ' '.join(valid_words)
    return a_string

test_string = 'I played and started playing with players and we all love to play with plays'
test_string = stem_words(test_string)
print(test_string)

i play and start play with player and we all love to play with play


In [69]:
# 5. MANDATORY: Make a pipeline function that applys all the text processing functions you just built.
def pipeline_function(a_string):
    a_string = make_lower(a_string)
    a_string = remove_punctuation(a_string)
    a_string = remove_stopwords(a_string)
    a_string = stem_words(a_string)
    return a_string

test_string = 'I played and started playing with players and we all love to play with plays'
test_string = pipeline_function(test_string)
print(test_string)

play start play player love play play


In [70]:
# 6. Mandatory: Use `df[column].apply(???)` and apply the text pipeline to your text data column. 
df['message_clean'] = df['message'].apply(pipeline_function)

In [71]:
df.head()
print("ORIGINAL TEXT:", df['message'][1])
print("CLEANDED TEXT:", df['message_clean'][1])

ORIGINAL TEXT: From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>

CLEANDED TEXT: guykuocarsonuwashingtonedu guy kuo subject si clock poll final call summari final call si clock report keyword 

# Text Vectorization

1. Define your `X` and `y` data. 


2. Initialize a vectorizer (you can use TFIDF or BOW, it is your choice).
    * Do you want to use n-grams..?


3. Fit your vectorizer using your X data.
    * Remember, this process happens IN PLACE.


4. Transform your X data using your fitted vectorizer. 
    * `X = vectorizer.???`



5. Print the shape of your X.  How many features (aka columns) do you have?

In [81]:
# 1. Define your `X` and `y` data. 
X = df['message_clean'].values

y = df['topic_category'].values


In [83]:
# 2. Initialize a vectorizer (you can use TFIDF or BOW, it is your choice).
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#X_train_text = X_train
#X_test_train = X_test

vectorizer = TfidfVectorizer()


In [84]:
# 3. Fit your vectorizer using your X data


vectorizer.fit(X_train)


TfidfVectorizer()

In [85]:
# 4. Transform your X data using your fitted vectorizer. 
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)


In [86]:
# 5. Print the shape of your X.  How many features (aka columns) do you have?

print(X_train.shape, type(X))

features = vectorizer.get_feature_names()
weights = vectorizer.idf_
print(len(features),len(weights))

df_idf = pd.DataFrame.from_dict({'feature':features, 'idf' : weights})

df_idf = df_idf.sort_values(by='idf', ascending = False)

df_idf

(9051, 105630) <class 'numpy.ndarray'>
105630 105630


C:\Users\conor\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,feature,idf
52815,knowladg,9.417594
62336,mbq72pl5u7tl2pqs7r_35iiquak1doqxqiqyi,9.417594
62354,mbrqmdq9i76igi086kopgg4x6zj5aoke2wplm,9.417594
62353,mbrllfnah,9.417594
62352,mbrian,9.417594
...,...,...
21037,articl,1.822709
102893,write,1.616816
76741,organ,1.038399
54897,line,1.003098


# Split your data into Training and Testing data. 

In [87]:
# Split our data into testing and training like always. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train_text = X_train
X_test_train = X_test

___
# Build and Train Model
Use Multinomial Naive Bayes to classify these documents. 

1. Initalize an empty model. 
2. Fit the model with our training data.


Experiment with different alphas.  Use the alpha gives you the best result.

EXTRA CREDIT:  Use grid search to programmatically do this for you. 

In [91]:
# 1. Initalize an empty model. 
model = MultinomialNB(alpha = 0.5)
model.fit(X_train,y_train)

ValueError: could not convert string to float: 'rupindangdartmouthedu rupin dang subject nikon fm2 len forsal organ dartmouth colleg hanov nh line 5 nikon fm2n 50 mm nikkor accessori salei bought camera hong kong two year ago everyth look well im sell gear financ next big film project ask 350 packag bargain'

In [89]:
X_train[0]

'rupindangdartmouthedu rupin dang subject nikon fm2 len forsal organ dartmouth colleg hanov nh line 5 nikon fm2n 50 mm nikkor accessori salei bought camera hong kong two year ago everyth look well im sell gear financ next big film project ask 350 packag bargain'

In [90]:
# Fit our model with our training data.



ValueError: could not convert string to float: 'rupindangdartmouthedu rupin dang subject nikon fm2 len forsal organ dartmouth colleg hanov nh line 5 nikon fm2n 50 mm nikkor accessori salei bought camera hong kong two year ago everyth look well im sell gear financ next big film project ask 350 packag bargain'

# Evaluate the model.

1. Make new predicitions using our test data. 
2. Print the accuracy of the model. 
3. Print the confusion matrix of our predictions. 
4. Using `classification_report` print the evaluation results for all the classes. 



In [ ]:
# 1. Make new predictions of our testing data. 

y_pred = model.predict(X_test)



In [ ]:
# 2. Print the accuracy of the model. 
accuracy = ???

print("Model Accuracy: %f" % accuracy)

In [ ]:
# 3. Plot the confusion matrix of our predictions



In [ ]:
# 4. Using `classification_report` print the evaluation results for all the classes. 



# Manual predicition
Write a new sentence that you think will be classified as talk.politics.guns. 
1. Apply the text pipeline to your sentence
2. Transform your cleaned text using the `X = vectorizer.transform([your_text])`
    * Note, the `transform` function accepts a list and not a individual string.
3. Use the model to predict your new `X`. 
4. Print the prediction

In [ ]:
my_sentence = ???

# 1. Apply the text pipeline to your sentence

# 2. Transform your cleaned text using the `X = vectorizer.transform([your_text])`\

# 3. Use the model to predict your new `X`. 

# 4. Print the prediction


___
# PART 2: Twitter Data
This part of the exercise is un-guided on purpose.  

Using the `dem-vs-rep-tweets.csv` build a classifier to determine if a tweet was written by a democrat or republican. 

Can you get an f1-score higher than %82

In [ ]:
# 1. Load the 20-newsgroups.csv data into a dataframe.
# 2. Print the shape
df = pd.read_csv('data/dem-vs-rep-tweets.csv')


